In [ ]:
import torch
import numpy as np
device = 'cuda'


: 

In [10]:
n=614
alpha = torch.randn(n).to('cuda',dtype=torch.float64)

In [39]:
J = torch.randn(n,532609).to('cuda',dtype=torch.float64)
# J = torch.concat([J, J+1e-4*torch.randn(256,532609)], dim=0)
K = torch.matmul(J, J.T)
# U, Lambda2, Vh = torch.linalg.svd(K)
Lambda2, U= torch.linalg.eigh(K)
print('shape check: ', J.shape, alpha.shape, K.shape, U.shape, Lambda2.shape)
print('condition number: ', torch.max(Lambda2)/torch.min(Lambda2))
print('rank check: ', torch.sum(Lambda2>1e-4))

shape check:  torch.Size([153, 532609]) torch.Size([153]) torch.Size([153, 153]) torch.Size([153, 153]) torch.Size([153])
condition number:  tensor(1.0699, device='cuda:0', dtype=torch.float64)
rank check:  tensor(153, device='cuda:0')


In [12]:
J = torch.load('Check_J.pt').to('cuda',dtype=torch.float64)
loss_scale = float(np.load('loss_scale.npy'))
K = torch.matmul(J, J.T)#.to(dtype=torch.float64)
U, Lambda2, Vh = torch.linalg.svd(K)
# Lambda2, U= torch.linalg.eigh(K)
print('shape check: ', J.shape, alpha.shape, K.shape, U.shape, Lambda2.shape)
print('condition number: ', torch.max(Lambda2)/torch.min(Lambda2))
print('rank check: ', torch.sum(Lambda2>1e-4))
J = J*(1/loss_scale)
K = K*(1/loss_scale)**2

shape check:  torch.Size([614, 299137]) torch.Size([614]) torch.Size([614, 614]) torch.Size([614, 614]) torch.Size([614])
condition number:  tensor(2.3302e+15, device='cuda:0', dtype=torch.float64)
rank check:  tensor(456, device='cuda:0')


In [13]:
uTa = (U.mH @ alpha)
# Solved_true = torch.linalg.solve(K@K, K@alpha)
Solved_true = torch.linalg.solve(torch.matmul(K,K), torch.matmul(K,alpha.reshape(-1,1)))
Solved_true = (J.T@Solved_true).reshape(-1,1)
criterion = (U.T@(J@Solved_true)).reshape(-1)/(uTa)
print("Criterion check: \nMean: ", torch.mean(criterion), "\nStd: ", torch.std(criterion), "\nABS Max: ", torch.max(torch.abs(criterion)), "\nABS Min: ", torch.min(torch.abs(criterion)))
print("Criterion check: ", criterion[:10], torch.sum(criterion>0.5))

Criterion check: 
Mean:  tensor(0.6267, device='cuda:0', dtype=torch.float64) 
Std:  tensor(3.0935, device='cuda:0', dtype=torch.float64) 
ABS Max:  tensor(64.5720, device='cuda:0', dtype=torch.float64) 
ABS Min:  tensor(1.8032e-08, device='cuda:0', dtype=torch.float64)
Criterion check:  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000], device='cuda:0', dtype=torch.float64) tensor(407, device='cuda:0')


In [7]:
print('Max: ',  torch.max(J), torch.max(torch.randn(*J.shape, device=device)))
print('Min: ', torch.min(J), torch.min(torch.randn(*J.shape, device=device)))
print('Mean: ', torch.mean(J), torch.mean(torch.randn(*J.shape, device=device)))
print('Std: ', torch.std(J), torch.std(torch.randn(*J.shape, device=device)))
torch.max(torch.abs(J),dim=0)

Max:  tensor(1.2984, device='cuda:0', dtype=torch.float64) tensor(5.8087, device='cuda:0')
Min:  tensor(-0.7252, device='cuda:0', dtype=torch.float64) tensor(-5.9129, device='cuda:0')
Mean:  tensor(-0.0023, device='cuda:0', dtype=torch.float64) tensor(7.2858e-05, device='cuda:0')
Std:  tensor(0.0257, device='cuda:0', dtype=torch.float64) tensor(1.0000, device='cuda:0')


torch.return_types.max(
values=tensor([0.0030, 0.0434, 0.0027,  ..., 0.0024, 0.0000, 1.0000], device='cuda:0',
       dtype=torch.float64),
indices=tensor([436, 351, 328,  ..., 448,   0,   0], device='cuda:0'))

In [ ]:
K = torch.matmul(J, J.T)

In [ ]:

for i in range(1,14):
    newK = K*(1/loss_scale*i)**2
    newJ = J*(1/loss_scale*i)  # torch.linalg.norm(J@J.T-K): 10 for 0., 12 for 11.51, 13 for 31.89
    print(torch.max(newJ), torch.min(newJ), torch.mean(newJ), torch.std(newJ))
    print(f"error {i}: ", torch.linalg.norm(torch.matmul(newJ, newJ.T)-newK))
# newK = K*(1/loss_scale*12.)**2
# newJ = J*(1/loss_scale*12.)  # torch.linalg.norm(J@J.T-K): 10 for 0., 12 for 11.51, 13 for 31.89

# print(f"error {i}: ", torch.linalg.norm(torch.matmul(newJ, newJ.T)-newK))

In [ ]:

for i in range(1,14):
    newK = K*(1/loss_scale*i)**2
    newJ = J*(1/loss_scale*i)  # torch.linalg.norm(J@J.T-K): 10 for 0., 12 for 11.51, 13 for 31.89
    # print(torch.max(newJ), torch.min(newJ), torch.mean(newJ), torch.std(newJ))
    print(f"error {i}: ", torch.linalg.norm(torch.matmul(newJ, newJ.T)-newK))
# newK = K*(1/loss_scale*12.)**2
# newJ = J*(1/loss_scale*12.)  # torch.linalg.norm(J@J.T-K): 10 for 0., 12 for 11.51, 13 for 31.89

# print(f"error {i}: ", torch.linalg.norm(torch.matmul(newJ, newJ.T)-newK))